In [2]:
from pathlib import Path
import pandas as pd

In [65]:
PWD = Path().cwd()
DATA = Path("../data")
IPS = DATA / "rnaseq_res_ips"
HIMORNA = DATA / "himorna_peaks"

parent_dir = PWD.parent
while par}ent_dir.joinpath(".git").exists() == False:
    print(parent_dir)
    parent_dir = parent_dir.parent
git_root = parent_dir

In [84]:
#Исходная таблица довольно большая. Будем работать с подмножеством

In [77]:
to_ips = IPS / "EdgeR.table.all.target.tsv.gz"
ips_expr = pd.read_csv(to_ips, sep="\t")
ips_expr.head()

,DEG.geneID,DEG.HGNC,EdgeR.log2FC,EdgeR.log2CPM,EdgeR.pvalue,EdgeR.FDR,EdgeR.Zscore,target.gapmerID,target.geneID,target.HGNC
0,CATG00000000155,CATG00000000155.1,0.257141,3.591144,0.550588,1.0,0.620927,G0234787_01,ENSG00000234787,LINC00458
1,CATG00000001164,CATG00000001164.1,0.706131,3.805757,0.044323,1.0,1.757448,G0234787_01,ENSG00000234787,LINC00458
2,CATG00000001318,CATG00000001318.1,-0.074655,4.766566,0.913694,1.0,0.052701,G0234787_01,ENSG00000234787,LINC00458
3,CATG00000002817,CATG00000002817.1,-0.201767,3.841792,0.716487,1.0,-0.414103,G0234787_01,ENSG00000234787,LINC00458
4,CATG00000003248,CATG00000003248.1,-0.155609,4.592817,0.650460,1.0,0.026003,G0234787_01,ENSG00000234787,LINC00458


In [69]:
to_himorna = HIMORNA / "Himorna_all_peaks_with_correlations_ensembl_id.tsv.gz"
himorna_peaks = pd.read_csv(
        to_himorna,
        chunksize=1000000,
        sep="\t",
        names=["hm", "lnc", "lnc_ens", "peak", "chrom", "start", "end", "gene", "gene_ens", "corr"]
    )
himorna_peaks = pd.concat(himorna_peaks)

/tmp/ipykernel_21886/1616637756.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  himorna_peaks = pd.concat(himorna_peaks)


In [85]:
himorna_peaks.head()

,hm,lnc,lnc_ens,peak,chrom,start,end,gene,gene_ens,corr
0,H3K9me3,AL390961.3,ENSG00000279212,peak_391,chr1,6203922,6205912,RPL22,ENSG00000116251,-0.540072
1,H3K9me3,SUGT1-DT,ENSG00000273723,peak_391,chr1,6203922,6205912,RPL22,ENSG00000116251,-0.493172
2,H3K9me3,AC004148.1,ENSG00000263272,peak_391,chr1,6203922,6205912,RPL22,ENSG00000116251,-0.406290
3,H3K9me3,CADM3-AS1,ENSG00000225670,peak_391,chr1,6203922,6205912,RPL22,ENSG00000116251,0.556495
4,H3K9me3,PCAT19,ENSG00000267107,peak_391,chr1,6203922,6205912,RPL22,ENSG00000116251,0.473327


In [89]:
meta = {
    "ips_expr": {"df": ips_expr,
                 "to_save": IPS / f"EdgeR.table.all.target_sample_{frac}_.tsv.gz",
                 "groupby": "target.geneID",
                 "frac": 0.1},
    
    "himorna_peaks": {"df": himorna_peaks,
                      "to_save": HIMORNA / f"Himorna_all_peaks_with_correlations_ensembl_id_{frac}_.tsv.gz",
                      "groupby": "hm",
                      "frac": 0.01}
}

In [ ]:
for name in meta:
    print(f"{name}")
    df_sample = []
    for key, subdf in meta[name]["df"].groupby(by=meta[name]["groupby"]):
        df_sample.append(subdf.sample(frac=meta[name]["frac"], random_state=30))
    df_sample = pd.concat(df_sample)
    print(f"\t{round(df_sample.shape[0] / meta[name]['df'].shape[0] * 100, 2)} %")
    df_sample.to_csv(meta[name]["to_save"], sep="\t", compression="gzip", index=False)

ips_expr
	10.0 %
himorna_peaks
